In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('data/enriched_bike_data.csv')
print(df.head())
print(df.columns)

                   timestamp  numberBike  is_weekend  is_holiday  \
0  2023-08-29 10:15:38+00:00         8.0       False       False   
1  2023-08-29 10:16:05+00:00         8.0       False       False   
2  2023-08-29 10:17:04+00:00         8.0       False       False   
3  2023-08-29 10:18:05+00:00         8.0       False       False   
4  2023-08-29 10:19:05+00:00         8.0       False       False   

   is_school_vacation  day_of_week  temp  max_temp  min_temp  precipitation  \
0                True            2  22.5      27.4      18.5            0.0   
1                True            2  22.5      27.4      18.5            0.0   
2                True            2  22.5      27.4      18.5            0.0   
3                True            2  22.5      27.4      18.5            0.0   
4                True            2  22.5      27.4      18.5            0.0   

   wind_speed  visibility  fog  rain  snow  hail  thunder  tornado  
0         7.9        11.9    0     0     0     

In [2]:

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

X = df.drop(['numberBike', 'timestamp'], axis=1)
y = df['numberBike']
date = df['timestamp']

# Ajout de l'heure et des minutes
X['hour'] = pd.to_datetime(date).dt.hour
X['minute'] = pd.to_datetime(date).dt.minute
X['hour'] = X['hour'].astype(int)
X['minute'] = X['minute'].astype(int)


X['day_of_week'] = X['day_of_week'].astype(int)

boolean_columns = ['is_weekend', 'is_holiday', 'is_school_vacation', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado']
for col in boolean_columns:
    X[col] = X[col].astype(int)

X = X.astype(float)

print("Forme des données d'entrée:", X.shape)
print("Forme des données cibles:", y.shape)
print("Colonnes des données d'entrée:", X.columns)
print("Types des colonnes:", X.dtypes)



Forme des données d'entrée: (40000, 18)
Forme des données cibles: (40000,)
Colonnes des données d'entrée: Index(['is_weekend', 'is_holiday', 'is_school_vacation', 'day_of_week', 'temp',
       'max_temp', 'min_temp', 'precipitation', 'wind_speed', 'visibility',
       'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'hour', 'minute'],
      dtype='object')
Types des colonnes: is_weekend            float64
is_holiday            float64
is_school_vacation    float64
day_of_week           float64
temp                  float64
max_temp              float64
min_temp              float64
precipitation         float64
wind_speed            float64
visibility            float64
fog                   float64
rain                  float64
snow                  float64
hail                  float64
thunder               float64
tornado               float64
hour                  float64
minute                float64
dtype: object


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

print("Forme des tenseurs d'entraînement:", X_train_tensor.shape, y_train_tensor.shape)
print("Forme des tenseurs de test:", X_test_tensor.shape, y_test_tensor.shape)

Forme des tenseurs d'entraînement: torch.Size([32000, 18]) torch.Size([32000, 1])
Forme des tenseurs de test: torch.Size([8000, 18]) torch.Size([8000, 1])


# Avec normalisation des valeurs

In [4]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# X = df.drop(['numberBike', 'timestamp'], axis=1)
# y = df['numberBike']
# date = df['timestamp']

# # Ajout de l'heure et des minutes
# X['hour'] = pd.to_datetime(date).dt.hour
# X['minute'] = pd.to_datetime(date).dt.minute

# # Assurez-vous que 'day_of_week' est traité comme un nombre
# X['day_of_week'] = X['day_of_week'].astype(int)

# # Normalisation de toutes les colonnes
# scaler = StandardScaler()
# X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# print("Forme des données d'entrée:", X_scaled.shape)
# print("Forme des données cibles:", y.shape)
# print("Colonnes des données d'entrée:", X_scaled.columns)

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# X_train_tensor = torch.FloatTensor(X_train.values)
# y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
# X_test_tensor = torch.FloatTensor(X_test.values)
# y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

# print("Forme des tenseurs d'entraînement:", X_train_tensor.shape, y_train_tensor.shape)
# print("Forme des tenseurs de test:", X_test_tensor.shape, y_test_tensor.shape)

# Création du model

In [5]:
from BikeNumberPredictor import BikeNumberPredictor, AsymmetricMSELoss

input_dim = X_train_tensor.shape[1]
model = BikeNumberPredictor(input_dim)
print(model)

criterion = AsymmetricMSELoss(beta=1.2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

BikeNumberPredictor(
  (layer1): Linear(in_features=18, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=32, bias=True)
  (layer3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


# Entraînement du modèle

In [7]:
y_train_adjusted = y_train * 0.8 # reduire la valeur de y_train pour eviter les erreurs de surestimation mieux vaut predire moins de velo que plus
num_epochs = 1000
batch_size = 64

if(num_epochs==0):
    model.load_state_dict(torch.load('model.pth'))

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Entraînement terminé!')

Epoch [10/1000], Loss: 0.3890
Epoch [20/1000], Loss: 0.3233
Epoch [30/1000], Loss: 0.3316
Epoch [40/1000], Loss: 0.3031
Epoch [50/1000], Loss: 0.3010
Epoch [60/1000], Loss: 0.2946
Epoch [70/1000], Loss: 0.2699
Epoch [80/1000], Loss: 0.2905
Epoch [90/1000], Loss: 0.2737
Epoch [100/1000], Loss: 0.2838
Epoch [110/1000], Loss: 0.2637
Epoch [120/1000], Loss: 0.2544
Epoch [130/1000], Loss: 0.2769
Epoch [140/1000], Loss: 0.2568
Epoch [150/1000], Loss: 0.2625
Epoch [160/1000], Loss: 0.2657
Epoch [170/1000], Loss: 0.2782
Epoch [180/1000], Loss: 0.2370
Epoch [190/1000], Loss: 0.2426
Epoch [200/1000], Loss: 0.2591
Epoch [210/1000], Loss: 0.2708
Epoch [220/1000], Loss: 0.2420
Epoch [230/1000], Loss: 0.2509
Epoch [240/1000], Loss: 0.2766
Epoch [250/1000], Loss: 0.2692
Epoch [260/1000], Loss: 0.2467
Epoch [270/1000], Loss: 0.2715
Epoch [280/1000], Loss: 0.2588
Epoch [290/1000], Loss: 0.3043
Epoch [300/1000], Loss: 0.2801
Epoch [310/1000], Loss: 0.2580
Epoch [320/1000], Loss: 0.3231
Epoch [330/1000],

# Évaluation du modèle

In [8]:
import numpy as np

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Erreur quadratique moyenne sur l\'ensemble de test: {test_loss.item():.4f}')

num_samples = min(10000, len(X_test))

indices = np.random.choice(len(X_test), num_samples, replace=False)

errors = []
max_error = 0
max_error_actual = 0
max_error_predicted = 0
errors_greater_than_one = 0
errors_less_than_one = 0

for i in indices:
    actual = y_test.iloc[i]
    predicted = y_pred[i].item()
    error = abs(actual - predicted)
    errors.append(error)
    
    if error > max_error:
        max_error = error
        max_error_actual = actual
        max_error_predicted = predicted
    
    if error > 1:
        errors_greater_than_one += 1
    else:
        errors_less_than_one += 1

mean_absolute_error = sum(errors) / len(errors)

print(f"\nNombre d'échantillons testés: {num_samples}")
print(f"Erreur absolue moyenne: {mean_absolute_error:.2f}")
print(f"Erreur maximale: {max_error:.2f}")
print(f"  Valeur réelle: {max_error_actual:.2f}")
print(f"  Valeur prédite: {max_error_predicted:.2f}")
print(f"Nombre d'erreurs supérieures à 1: {errors_greater_than_one}")
print(f"Nombre d'erreurs inférieures ou égales à 1: {errors_less_than_one}")
print(f"Pourcentage d'erreurs supérieures à 1: {(errors_greater_than_one / num_samples) * 100:.2f}%")

print("\nQuelques exemples de prédictions:")
num_examples = 5
example_indices = np.random.choice(len(indices), num_examples, replace=False)
for i in example_indices:
    actual = y_test.iloc[indices[i]]
    predicted = y_pred[indices[i]].item()
    print(f'Valeur réelle: {actual:.2f}, Valeur prédite: {predicted:.2f}, Erreur: {abs(actual - predicted):.2f}')

Erreur quadratique moyenne sur l'ensemble de test: 0.2681

Nombre d'échantillons testés: 8000
Erreur absolue moyenne: 0.38
Erreur maximale: 3.58
  Valeur réelle: 8.00
  Valeur prédite: 4.42
Nombre d'erreurs supérieures à 1: 428
Nombre d'erreurs inférieures ou égales à 1: 7572
Pourcentage d'erreurs supérieures à 1: 5.35%

Quelques exemples de prédictions:
Valeur réelle: 5.00, Valeur prédite: 4.83, Erreur: 0.17
Valeur réelle: 6.00, Valeur prédite: 5.75, Erreur: 0.25
Valeur réelle: 6.00, Valeur prédite: 5.68, Erreur: 0.32
Valeur réelle: 4.00, Valeur prédite: 3.94, Erreur: 0.06
Valeur réelle: 6.00, Valeur prédite: 5.64, Erreur: 0.36


In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
print(input_dim)